<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [1]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [2]:
# parte (a)
df1 = pd.read_csv(archivos_anio[0])
df2 = pd.read_csv(archivos_anio[1])

df1.columns = df1.columns.str.lower()
df2.columns = df2.columns.str.lower()

df_anio = pd.concat([df1,df2])

df_anio.head(10)

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN
5,ARG,2001,12.0,8.0
6,ARM,2001,NaN,NaN
7,ATG,2001,NaN,NaN
8,AUS,2001,3.5,48.0
9,AUT,2001,7.5,86.0


In [3]:
# parte (b)
# agrupamos por codigo_iso contando cuántos países tienen asociados
duplicados = df_codigos.groupby("codigo_iso")["pais"].nunique()
# buscamos los codigo_iso que contienen más de una entrada de un país
duplicados = duplicados[duplicados == 2]
duplicados
# Eliminamos el país asociado inválido
df_codigos = df_codigos[(df_codigos['pais']!='malo')]


In [4]:
# parte (c)
# juntar df's objetivos
df = pd.merge(df_anio,df_codigos, on = 'codigo_iso')



### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [5]:
# Recopilación de información
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB


El Data Frame consolidado cuenta con 3060 filas y 5 columnas.

Las columnas tienen los nombres `codigo_iso`, `anio`, `indice`, `ranking` y `pais`; con los tipos de datos texto, entero, flotante, flotante y texto respectivamente.

Todas las columnas parecen tener un tipo de dato esperado.

In [6]:
df.describe()

,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.782316,477.930913
std,5.786024,2695.525264,6474.935347
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,34.000000
50%,2009.000000,28.000000,70.000000
75%,2015.000000,41.227500,110.000000
max,2019.000000,64536.000000,121056.000000


En los valores obtenidos en el resumen estadístico, los valores de `indice` y `ranking` mínimos son 0.0 y 1.0 respectivamente, y sus valores máximos son 64536.0 y 121056.0 respectivamente.

Además podemos encontrar un promedio mundial de 205.78 de índice de libertad de prensa y un 477.93 de ranking mundial

In [7]:
df_indice_menor = df[(df['indice']==df["indice"].min())]
df_indice_mayor = df[(df['indice']==df['indice'].max())]
df_ranking_menor = df[(df['ranking']==df['ranking'].min())]
df_ranking_mayor = df[(df['ranking']==df['ranking'].max())]

Los países con menor índice de libertad de prensa son: Dinamarca en 2008, Finlandia en 2008 y 2009, Irlanda en 2008, Noruega en 2008 y 2009, Suecia en 2008 y 2009, Suiza en 2009, Islandia en 2009, Países Bajos en 2009.

Mientras que el país con mayor índice de libertad de prensa es: Kosovo en 2014.

Luego, los países con menor ranking mundial de libertad de prensa son: Finlandia desde 2001 hasta 2005, Islandia desde 2001 hasta 2007, Países Bajos desde 2001 hasta 2005, Noruega desde 2001 hasta 2004, desde 2006 hasta 2007 y desde 2017 hasta 2019, Suiza desde 2003 hasta 2004, Dinamarca desde 2003 hasta 2004 y 2016, Irlanda desde 2003 hasta 2005 y 2012, Eslovaquia en 2003, Luxemburgo en 2007, Estonia en 2008, Austria en 2009 y 2013 y Nueva Zelanda desde 2014 hasta 2015.

Mientras que el país con mayor ranking mundial de libertad de prenssa es: Kosovo en 2015.

In [8]:
df.isna().sum()

,0
codigo_iso,0
anio,0
indice,396
ranking,223
pais,0


Podemos encontrar la siguiente cantidad de nulos por columna:
* Columna `codigo_iso`: 0 nulos
* Columna `anio`: 0 nulos
* Columna `indice`: 396 nulos
* Columna `ranking`: 223 nulos
* Columna `pais`: 0 nulos

Por lo que las columnas `codigo_iso`, `anio` y `pais` tienen un 0% de nulos, la columna `indice` un 12.05% de datos nulos y la columna de `ranking` un 7.28% de datos nulos; por lo tanto, no hay ninguna columna que contenga más del 30% de datos faltantes.





In [9]:
pais_unico = df["pais"].nunique()
anio_unico = df['anio'].nunique()
cant_duplicados = df.duplicated().sum()
print("Hay ", pais_unico, " cantidad de países únicos en el DataFrame")
print("Hay ", anio_unico, " cantidad de años distintos representados en el DataFrame")
print("Hay ", cant_duplicados, " cantidad de filas duplicadas (exactamente iguales)")

Hay  179  cantidad de países únicos en el DataFrame
Hay  17  cantidad de años distintos representados en el DataFrame
Hay  0  cantidad de filas duplicadas (exactamente iguales)


In [10]:
inconsistencias = df.groupby("codigo_iso")["pais"].nunique()
inconsistencias = inconsistencias[inconsistencias == 2].value_counts()
print("La cantidad de inconsistencias en el DataFrame es: ", inconsistencias.shape[0])

La cantidad de inconsistencias en el DataFrame es:  0





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [11]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

# Parte (a)
df_america =  pd.DataFrame()
df_america = df[df["codigo_iso"].isin(america)]
df_anio = pd.DataFrame
a_indice_menor = pd.DataFrame(columns=["codigo_iso", "pais", "anio", "indice"])
a_indice_mayor = pd.DataFrame(columns=["codigo_iso", "pais", "anio", "indice"])
for a in df_america["anio"].unique():
    df_anio = df_america[df_america['anio'] == a]
    a_indice_menor = pd.concat([a_indice_menor, df_anio[df_anio["indice"] == df_anio["indice"].min()]], ignore_index=True)
    a_indice_mayor = pd.concat([a_indice_mayor, df_anio[df_anio["indice"] == df_anio["indice"].max()]], ignore_index=True)

/tmp/ipython-input-181998293.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  a_indice_menor = pd.concat([a_indice_menor, df_anio[df_anio["indice"] == df_anio["indice"].min()]], ignore_index=True)
/tmp/ipython-input-181998293.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  a_indice_mayor = pd.concat([a_indice_mayor, df_anio[df_anio["indice"] == df_anio["indice"].max()]], ignore_index=True)


In [12]:
print("Así, los países con mayor libertad de prensa por año son")
a_indice_menor

Así, los países con mayor libertad de prensa por año son


,codigo_iso,pais,anio,indice,ranking
0,CAN,Canadá,2001,0.80,2.0
1,TTO,Trinidad y Tobago,2002,1.00,2.0
2,TTO,Trinidad y Tobago,2003,2.00,30.0
3,TTO,Trinidad y Tobago,2004,2.00,31.0
4,BOL,Bolivia,2005,4.50,63.0
5,CAN,Canadá,2005,4.50,63.0
6,CAN,Canadá,2006,4.88,84.0
7,CAN,Canadá,2007,3.33,50.0
8,CAN,Canadá,2008,3.70,62.0
9,USA,Estados Unidos,2009,6.75,115.0


In [13]:
print("Así, los países con menor libertad de prensa por año son")
a_indice_mayor

Así, los países con menor libertad de prensa por año son


,codigo_iso,pais,anio,indice,ranking
0,CUB,Cuba,2001,90.30,99.0
1,CUB,Cuba,2002,97.83,125.0
2,ARG,Argentina,2003,35826.00,35.0
3,CUB,Cuba,2004,87.00,112.0
4,CUB,Cuba,2005,95.00,109.0
5,CUB,Cuba,2006,96.17,139.0
6,CUB,Cuba,2007,88.33,117.0
7,CUB,Cuba,2008,94.00,131.0
8,CUB,Cuba,2009,78.00,129.0
9,CUB,Cuba,2012,71.64,162.0


In [14]:
# parte (b)
df_america['flag_menor'] = df_america.groupby('anio')['indice'].transform('min')
df_america['flag_mayor'] = df_america.groupby('anio')['indice'].transform('max')
b_indice_menor = df_america[df_america['indice']==df_america['flag_menor']]
b_indice_mayor = df_america[df_america['indice']==df_america['flag_mayor']]

/tmp/ipython-input-1417934327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_america['flag_menor'] = df_america.groupby('anio')['indice'].transform('min')
/tmp/ipython-input-1417934327.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_america['flag_mayor'] = df_america.groupby('anio')['indice'].transform('max')


In [15]:
print("Así, los países con mayor libertad de prensa por año son")
a_indice_menor

Así, los países con mayor libertad de prensa por año son


,codigo_iso,pais,anio,indice,ranking
0,CAN,Canadá,2001,0.80,2.0
1,TTO,Trinidad y Tobago,2002,1.00,2.0
2,TTO,Trinidad y Tobago,2003,2.00,30.0
3,TTO,Trinidad y Tobago,2004,2.00,31.0
4,BOL,Bolivia,2005,4.50,63.0
5,CAN,Canadá,2005,4.50,63.0
6,CAN,Canadá,2006,4.88,84.0
7,CAN,Canadá,2007,3.33,50.0
8,CAN,Canadá,2008,3.70,62.0
9,USA,Estados Unidos,2009,6.75,115.0


In [16]:
print("Así, los países con menor libertad de prensa por año son")
b_indice_mayor

Así, los países con menor libertad de prensa por año son


,codigo_iso,anio,indice,ranking,pais,flag_menor,flag_mayor
39,CUB,2001,90.30,99.0,Cuba,0.80,90.30
219,CUB,2002,97.83,125.0,Cuba,1.00,97.83
365,ARG,2003,35826.00,35.0,Argentina,2.00,35826.00
579,CUB,2004,87.00,112.0,Cuba,2.00,87.00
759,CUB,2005,95.00,109.0,Cuba,4.50,95.00
939,CUB,2006,96.17,139.0,Cuba,4.88,96.17
1119,CUB,2007,88.33,117.0,Cuba,3.33,88.33
1299,CUB,2008,94.00,131.0,Cuba,3.70,94.00
1479,CUB,2009,78.00,129.0,Cuba,6.75,78.00
1659,CUB,2012,71.64,162.0,Cuba,9.88,71.64


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [37]:
# Tarea principal
df_nuevo = df.pivot_table(index = 'pais', columns = 'anio', values = 'indice', fill_value = 0)
df_nuevo

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,81.3,89.17,86.88,73.25,67.25,79.25,86.17,81.67,75.75,71.78,72.36,72.63,74.27,73.96,75.05,74.93
West Bank y Gaza,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42.90,42.96,44.68
Yemen,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,65.80,62.23,61.66


In [38]:
# parte (a)
indice_max_global = df_nuevo.max().max()
pais_max_global = df_nuevo.stack().idxmax()[0]
indice_min_global = df_nuevo.min().min()
pais_min_global = df_nuevo.stack().idxmin()[0]
print("El país con mayor índice es ", pais_max_global, " con un índice de ", indice_max_global)
print("El país con mínimo índice es ", pais_min_global, " con un índice de ", indice_min_global)

El país con mayor índice es  Kosovo  con un índice de  64536.0
El país con mínimo índice es  Albania  con un índice de  0.0


In [39]:
promedios = df_nuevo.mean(axis=0)
prom_max = promedios.max()
anio_max = promedios.idxmax()
prom_min = promedios.min()
anio_min = promedios.idxmin()
print("Durante el año ", anio_max, " se obtuvo el promedio mayor de índice de libertad: ", prom_max)
print("Durante el año ", anio_min, " se obtuvo el promedio mínimo de índice de libertad: ", prom_min)

Durante el año  2013  se obtuvo el promedio mayor de índice de libertad:  449.0847206703911
Durante el año  2001  se obtuvo el promedio mínimo de índice de libertad:  20.024022346368714


In [43]:
# parte (c)
variabilidad = df_nuevo.max(axis=1) - df_nuevo.min(axis=1)
var_max = variabilidad.max()
pais_max_var = variabilidad.idxmax()
print("El país con mayor variabilidad es ", pais_max_var, " con una variabilidad de ", var_max)

El país con mayor variabilidad es  Kosovo  con una variabilidad de  64536.0


In [60]:
#parte (d)
'''
para esto utilizaremos el anterior, puesto que si
    índice_mayor - índice menor = 0
entonces su índice es constante
'''
constantes = variabilidad[variabilidad == 0].index
df_constantes = df_nuevo.loc[constantes]
df_constantes = df_constantes.iloc[:,[0]]
df_constantes = df_constantes.rename(columns = {df_constantes.columns[0]:'índice_constante'})
print("Los países con índice constantes son: ")
df_constantes

Los países con índice constantes son: 


anio,índice_constante
pais,


In [61]:
# parte (e)
sin_datos = df_nuevo[(df_nuevo == 0).all(axis=1)]
print("El listado de países sin datos son:")
sin_datos

El listado de países sin datos son:


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


Que algún país no tenga ningún tipo de dato puede ocurrir porque en los datos otorgados estos no contienen datos, por lo que termina sin ningún tipo de dato.

Otro caso que puede haber ocurrido es un error al reemplazar los datos nulos o al pivotear el DataFrame original, hacerlo de forma errónea.